In [1]:
#import libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import itertools
import datetime as dt
from scipy import stats
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
import os
import gc 
import pandas_profiling

In [2]:
test_bundas = pd.read_csv('C:/Users/jonathan/Desktop/ilabdata/bundas_test.csv')

In [3]:
# Checking Null values in test Set
test_bundas.isnull().sum()

Item_ID                        0
Weight                       976
FatContent                     0
Visibility                     0
Category                       0
Max_Price                      0
Store_ID                       0
Store_Establishment_Year       0
Store_Size                  1606
Store_Location_Type            0
Store_Type                     0
dtype: int64

In [4]:
from sklearn.preprocessing import Imputer

# Dealing with the NAs


We discovered there are blanks in the weight (976) collum and the store size (1606). After a deeper analysis we discovered that some of the missing entries for weights column have the same item idea as the products with the weight entries hence we decided to fill up the entries with blanks considering their item idea.  
  
colSums(is.na(ts))

ts$tracker<-1:nrow(ts)

## getting columns with the weight entries
drna<-ts[complete.cases(ts$Weight),]

tra<-drna$tracker

# the entries without any weight entries
dropd<-ts[-tra,]

## confirming the combined splits are the same as the original data
nrow(drna)+nrow(dropd)
nrow(ts)


colSums(is.na(drna))

weigts<-drna %>% 
  group_by(Item_ID,Weight) %>% 
  summarise(x=1)

colnames(weigts)<-c("Item_ID","Weight_al","x")
compwight<-left_join(dropd, weigts[,-3], by="Item_ID")

compwight.Weight<-compwight.Weight_al

In the end only 20 entries are still empty for the weights collumn
colSums(is.na((compwight)))

sdf<-compwight[complete.cases(compwight)==FALSE,]

## combining the two to create a new data set
cmw_ts<-rbind(drna,compwight[,-c(13)])

We decided to drop the entries that still have empty entries leading to loose of 28.62 % of the data 
fin_ts<-cmw_ts[complete.cases(cmw_ts),]
(1-nrow(fin_ts)/nrow(ts))*100

colSums(is.na(fin_ts))
write.csv(fin_ts,"comp_bundas_test.csv.csv")



In [7]:

df_mean_imputed = pd.read_csv('C:/Users/jonathan/Desktop/ilabdata/comp_bundas_test.csv')

In [8]:

df_mean_imputed.loc[df_mean_imputed['FatContent'] == 'LF', 'FatContent'] = 'Low Fat'
df_mean_imputed.loc[df_mean_imputed['FatContent'] == 'reg', 'FatContent'] = 'Regular'
df_mean_imputed.loc[df_mean_imputed['FatContent'] == 'low fat', 'FatContent'] = 'Low Fat'

In [9]:
df_mean_imputed['FatContent'].unique()


array(['Low Fat', 'Regular'], dtype=object)

In [10]:
# Checking Null values in test Set
df_mean_imputed.isnull().sum()

Item_ID                     0
Weight                      0
FatContent                  0
Visibility                  0
Category                    0
Max_Price                   0
Store_ID                    0
Store_Establishment_Year    0
Store_Size                  0
Store_Location_Type         0
Store_Type                  0
dtype: int64

In [11]:
modDf = df_mean_imputed.dropna(how='any', subset=['Store_Size'])

In [12]:
# One hot encode the categorical columns
FatContent_one_hot = pd.get_dummies(modDf['FatContent'], 
                                     prefix='FatContent')

Category_one_hot = pd.get_dummies(modDf['Category'], 
                                prefix='Category')

Store_Size_one_hot = pd.get_dummies(modDf['Store_Size'], 
                                   prefix='Store_Size')

# One hot encode the categorical columns
Store_Location_Type_one_hot = pd.get_dummies(modDf['Store_Location_Type'], 
                                     prefix='Store_Location_Type')

Store_Type_one_hot = pd.get_dummies(modDf['Store_Type'], 
                                prefix='Store_Type')

Store_ID_one_hot = pd.get_dummies(modDf['Store_ID'], 
                                   prefix='Store_ID')

In [13]:
# Combine the one hot encoded columns with df_con_enc
modDfenc = pd.concat([modDf, 
                        FatContent_one_hot, 
                        Category_one_hot, 
                        Store_Size_one_hot,
                        Store_Location_Type_one_hot, 
                        Store_Type_one_hot, 
                        Store_ID_one_hot], axis=1)

# Drop the original categorical columns (because now they've been one hot encoded)
modDfenc = modDfenc.drop(['FatContent','Category','Store_Size','Store_Location_Type','Store_Type','Store_ID'], axis=1)

In [14]:
# Seclect the dataframe we want to use first for predictions
selected_df = modDfenc

In [15]:
selected_df.to_csv('C:/Users/jonathan/Desktop/ilabdata/test_modified.csv',index=False)

In [15]:
selected_df.head()

,Item_ID,Weight,Visibility,Max_Price,Store_Establishment_Year,FatContent_Low Fat,FatContent_Regular,Category_Baking Goods,Category_Breads,Category_Breakfast,...,Store_Type_Supermarket Type1,Store_Type_Supermarket Type2,Store_Type_Supermarket Type3,Store_ID_OUT013,Store_ID_OUT018,Store_ID_OUT019,Store_ID_OUT027,Store_ID_OUT035,Store_ID_OUT046,Store_ID_OUT049
0,FDW58,20.750,0.007565,107.8622,1999,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,FDH56,9.800,0.063817,117.1492,1997,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,FDL48,19.350,0.082602,50.1034,2009,0,1,1,0,0,...,0,1,0,0,1,0,0,0,0,0
3,FDU11,4.785,0.092738,122.3098,1999,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
4,DRL59,16.750,0.021206,52.0298,1987,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
